# VST Sound with Pedalboard

Experimental notebook, used to understand pedalboard and to generate a single sound.

In [1]:
import random
from dataclasses import dataclass
from typing import Any, List, Tuple, Dict

import numpy as np
from pedalboard import VST3Plugin
from pedalboard.io import AudioFile
from pydub import AudioSegment
from pydub.playback import play
import matplotlib.pyplot as plt
from pyloudnorm import Meter
import librosa
import mido
import threading
import time
import _thread

## Core VST Functions

In [2]:
def _call_with_interrupt(fn, sleep_time: float = 2.0):
    def send_interrupt():
        time.sleep(sleep_time)
        _thread.interrupt_main()
    
    t = threading.Thread(target=send_interrupt)
    t.start()
    
    try:
        fn()
    except KeyboardInterrupt:
        print("Interrupted main thread.")
    finally:
        t.join()

def _prepare_plugin(plugin: VST3Plugin) -> None:
    _call_with_interrupt(plugin.show_editor, sleep_time=2.0)

def load_plugin(plugin_path: str) -> VST3Plugin:
    print(f"Loading plugin {plugin_path}")
    p = VST3Plugin(plugin_path)
    print("Plugin loaded, preparing...")
    _prepare_plugin(p)
    print("Plugin ready")
    return p

def load_preset(plugin: VST3Plugin, preset_path: str) -> None:
    print(f"Loading preset {preset_path}")
    plugin.load_preset(preset_path)
    print("Preset loaded")

def set_params(plugin: VST3Plugin, params: Dict[str, float]) -> None:
    for k, v in params.items():
        plugin.parameters[k].raw_value = v

def make_midi_events(pitch: int, velocity: int, note_start: float, note_end: float):
    events = []
    note_on = mido.Message("note_on", note=pitch, velocity=velocity, time=0)
    events.append((note_on.bytes(), note_start))
    note_off = mido.Message("note_off", note=pitch, velocity=velocity, time=0)
    events.append((note_off.bytes(), note_end))
    return tuple(events)

def render_params(
    plugin: VST3Plugin,
    params: Dict[str, float],
    midi_note: int,
    velocity: int,
    note_start_and_end: Tuple[float, float],
    signal_duration_seconds: float,
    sample_rate: float,
    channels: int,
    preset_path: str = None,
) -> np.ndarray:
    if preset_path is not None:
        load_preset(plugin, preset_path)
    plugin.process([], 32.0, sample_rate, channels, 2048, True)
    plugin.reset()
    set_params(plugin, params)
    plugin.process([], 32.0, sample_rate, channels, 2048, True)
    plugin.reset()
    midi_events = make_midi_events(midi_note, velocity, *note_start_and_end)
    output = plugin.process(midi_events, signal_duration_seconds, sample_rate, channels, 2048, True)
    return output

## Data Structures and Sampling Utilities

In [3]:
@dataclass
class VSTDataSample:
    audio: np.ndarray
    synth_params: Dict[str, float]
    sample_rate: float
    channels: int
    midi_note: int
    velocity: int
    note_start: float
    note_end: float
    duration: float

def random_synth_params(plugin: VST3Plugin, num_params: int = 5) -> Dict[str, float]:
    params = {}
    keys = list(plugin.parameters.keys())
    chosen = random.sample(keys, min(num_params, len(keys)))
    for k in chosen:
        params[k] = random.random()
    return params

def make_spectrogram(audio: np.ndarray, sample_rate: int, n_mels: int = 128):
    S = librosa.feature.melspectrogram(y=audio, sr=sample_rate, n_mels=n_mels)
    S_dB = librosa.power_to_db(S, ref=np.max)
    return S_dB

## Example Usage

In [ ]:
plugin_path = "/usr/lib64/vst3/Surge XT.vst3"
plugin = load_plugin(plugin_path)

Loading plugin plugins/surge_xt.vst3


ImportError: Unable to scan plugin plugins/surge_xt.vst3: unsupported plugin format or scan failure.

In [ ]:
params = random_synth_params(plugin, num_params=5)
output = render_params(
    plugin,
    params,
    midi_note=60,
    velocity=100,
    note_start_and_end=(0.0, 1.0),
    signal_duration_seconds=2.0,
    sample_rate=44100,
    channels=2
)

sample = VSTDataSample(
    audio=output,
    synth_params=params,
    sample_rate=44100,
    channels=2,
    midi_note=60,
    velocity=100,
    duration=2.0,
    note_start=0.0,
    note_end=2.0,
)

print("Generated sample with params:", sample.synth_params)

In [ ]:
import IPython.display as ipd
ipd.Audio(sample.audio, rate = sample.sample_rate)

## Spectrogram Plot

In [ ]:
print(np.max(sample.audio))

In [ ]:
mel_spec = make_spectrogram(sample.audio[:, 1], int(sample.sample_rate))
plt.figure(figsize=(12, 6))
plt.imshow(mel_spec, aspect='auto', origin='lower')
plt.colorbar(label='dB')
plt.xlabel('Time (frames)')
plt.ylabel('Mel bins')
plt.title('Mel Spectrogram of Generated Sample')
plt.show()

## Loudness Calculation

In [ ]:
meter = Meter(sample.sample_rate)
loudness = meter.integrated_loudness(sample.audio)
print("Integrated loudness (LUFS):", loudness)